In [1]:
import numpy as np
import os
import pandas as pd
from netCDF4 import Dataset
import pygplates
import pickle
import time as tme
import itertools
import MOR_characterisation, filter_and_reconstruct_points_PMSR, poly_processing#, points_spatial_tree, points_in_polygons
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.colors import LogNorm

/Users/Andrew/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1066: UserWarning: Duplicate key in file "/Users/Andrew/.matplotlib/matplotlibrc", line #3
  (fname, cnt))
/Users/Andrew/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1066: UserWarning: Duplicate key in file "/Users/Andrew/.matplotlib/matplotlibrc", line #4
  (fname, cnt))


In [2]:
loaddir = '/Users/andrew/Dropbox/1Mysteps_02042019/base_files_scripts'
savedir = '/Users/Andrew/Dropbox/1Mysteps_02042019/review_sandboxing/base_results/'

rotfile = '%s/Matthews_etal_GPC_2016_410-0Ma_GK07.rot' % loaddir
coastlines = '%s/Global_EarthByte_GeeK07_COB_Terranes_Matthews_etal.gpml' % loaddir
#coastlines = '%s/Global_coastlines_2015_v1_low_res_ASM.gpml' % loaddir
topologies = ['%s/Matthews_etal_GPC_2016_MesozoicCenozoic_PlateTopologies.gpmlz' % loaddir,
              '%s/Matthews_etal_GPC_2016_Paleozoic_PlateTopologies.gpmlz' % loaddir]

rotation_model = pygplates.RotationModel(rotfile)
#rotation_model_palaeozoic = pygplates.RotationModel(rotfile_palaeozoic)
coastlines = pygplates.FeatureCollection(coastlines)
polygons = []
for feature in coastlines:
    for geom in feature.get_geometries():
        polygon = feature
        polygon.set_geometry(pygplates.PolygonOnSphere(geom))
        polygons.append(polygon)

coastlines = pygplates.FeatureCollection(polygons)

In [3]:
# import previously made file of plate boundary data in 'extract velocities'
#separate load so faster if we need to recheck stuff
C_storage = pickle.load(open('%s/C_storage_2019-09-14_plate_model_velocity.p' % loaddir, 'rb'))

In [5]:
#setup parameters for plotting spreading rate
resolution = 1 #degrees
make_raster_at = -1
timestep = 1
times = np.arange(350,make_raster_at,-(timestep))
#print 'make_raster_at', make_raster_at

#inititate loop to extract data
start_time = tme.time()
for time in times:
    print time
    next_timestep = time - timestep
    previous_timestep = time + timestep
    #resolve topologies for rotating some points
    #reconstruct
    tmp_reconstructed_coastlines = []
    curr_resolved_topologies = []
    prev_resolved_topologies = []
    pygplates.reconstruct(coastlines, rotation_model, tmp_reconstructed_coastlines, time)
    pygplates.resolve_topologies(topologies, rotation_model, curr_resolved_topologies, time)
    pygplates.resolve_topologies(topologies, rotation_model,prev_resolved_topologies, previous_timestep)
    #get plate IDs of current points that have beend passed through, and not at time=0
    #incase they\'ve changed plate IDs (ergh)
    if time != np.max(times):
        #time = 100, next_timestep = 99
        #next becomes current, current becomes previous
        prev_lat, prev_lon = curr_lat, curr_lon
        curr_lat, curr_lon, curr_zvals = next_lat, next_lon, next_zvals
        #print 'here', len(curr_lat)
        prev_plate_partitioner, prev_plateIDs = curr_plate_partitioner, curr_plateIDs
        #get new \'curr_plate_partitioner\'
        curr_plate_partitioner = pygplates.PlatePartitioner(curr_resolved_topologies, rotation_model)
        #get new set of plate IDs for current timestep
        curr_plateIDs = filter_and_reconstruct_points_PMSR.get_plate_ID_of_points(curr_plate_partitioner,
                                                                             curr_lat,
                                                                             curr_lon)
        #returns none if it falls into a topology gap then filters to remove it
        curr_lat, curr_lon, curr_zvals = filter_and_reconstruct_points_PMSR.filter_points_Nones(curr_lat, 
                                                                                           curr_lon,
                                                                                           curr_plateIDs,
                                                                                           curr_zvals)
        #lets get the resolved boundares of our points to test against
        #there should be no \'Nones\' present, since we got rid of them just before
        #print prev_lat, prev_lon, prev_resolved_topologies
        print 'first', len(curr_lat)
        resolved_topologies_containing_prev_points = filter_and_reconstruct_points_PMSR.find_polygon_of_points(prev_lat,
                                                                            prev_lon,
                                                                            prev_resolved_topologies)
        #next_plateIDs = filter_and_reconstruct_points.get_plate_ID_of_points(next_plate_partitioner,lat,lon)
        #curr_X at time = 100
        #filter current points
        curr_lat = filter(None, curr_lat)
        curr_lon = filter(None, curr_lon)
        curr_zvals = filter(None, curr_zvals)
        curr_plateIDs = filter(None, curr_plateIDs)
        print 'second', len(curr_lat)
        for ind, i in enumerate(prev_plateIDs):
            #there are more plate IDs in current that previous, because we have all the new points attached
            #but because they are appended, they occur at the end, so indices of prev_plateIDs should correspond
            #to indices of curr_plateIDs until we run out of prev_plateIDs
            prev_inputlatLon = pygplates.LatLonPoint(prev_lat[ind], prev_lon[ind])
            prev_latLonPoint = pygplates.convert_lat_lon_point_to_point_on_sphere(prev_inputlatLon)
            curr_inputlatLon = pygplates.LatLonPoint(curr_lat[ind], curr_lon[ind])
            curr_latLonPoint = pygplates.convert_lat_lon_point_to_point_on_sphere(curr_inputlatLon)
            prev_point_rotation = rotation_model.get_rotation(time, i, previous_timestep)
            curr_point_rotation = rotation_model.get_rotation(time, curr_plateIDs[ind], previous_timestep)
            #(after Williams et al. in review)
            if curr_plateIDs[ind] != i:
                collision = filter_and_reconstruct_points_PMSR.detect_collision(timestep,
                                             curr_latLonPoint,
                                             prev_latLonPoint,
                                             curr_point_rotation,
                                             prev_point_rotation,
                                             curr_plateIDs[ind],
                                             resolved_topologies_containing_prev_points[ind],
                                             i)
                if collision == True:
                    #print 'h'
                    curr_lat[ind] = None
                    curr_lon[ind] = None
                    curr_zvals[ind] = None
                    curr_plateIDs[ind] = None
        curr_lat = filter(None, curr_lat)
        curr_lon = filter(None, curr_lon)
        curr_zvals = filter(None, curr_zvals)
        curr_plateIDs = filter(None, curr_plateIDs)
        print 'third', len(curr_lat)
        #now we can export our data
        if time < 330:
            filter_and_reconstruct_points_PMSR.exporting(curr_lon,curr_lat,curr_zvals, time, savedir)
           #exporting values to xyz table
        #now to rotate through time to next timestep
        #re-initiate the next time step lat/lon/zvals, as we want to overwrite all previous 'nexts'
        #(which are 'current')
        #with the actual next timestep
        next_lat = []
        next_lon = []
        next_zvals = []
        for ind, i in enumerate(curr_plateIDs):
            if i is not None:
                #print ind
                #we have already accounted for points that have been subducted, so lets rotate them through time to generate next
                #NB we output points for next timestep
                curr_inputlatLon = pygplates.LatLonPoint(curr_lat[ind], curr_lon[ind])
                curr_latLonPoint = pygplates.convert_lat_lon_point_to_point_on_sphere(curr_inputlatLon)
                next_point_rotation = rotation_model.get_rotation(next_timestep, i, time)
                next_reconstructed_point = next_point_rotation * curr_latLonPoint
                tmp_lat = next_reconstructed_point.to_lat_lon_list()[0][0]
                tmp_lon = next_reconstructed_point.to_lat_lon_list()[0][1]
                #print tmp_lat, tmp_lon, next_reconstructed_point
                next_lat.append(tmp_lat)
                next_lon.append(tmp_lon)
                next_zvals.append(curr_zvals[ind])
    
    #get new data
    MOR_lat = []
    MOR_lon = []
    MOR_zvals = []
    MOR_plateIDs = []
    init_curr_lat = []
    init_curr_lon = []
    init_curr_plateIDs = []
    for i,j,k,l,m,n,o,p,q,r,s,t,u in itertools.izip(C_storage[time]['Start-Lat-Point'],
                                                   C_storage[time]['End-Lat-Point'],
                                                   C_storage[time]['Start-Lon-Point'],
                                                   C_storage[time]['End-Lon-Point'],
                                                   C_storage[time]['RightPlate'],
                                                   C_storage[time]['LeftPlate'],
                                                   C_storage[time]['C-prod'],
                                                   C_storage[time]['C-prod_vertical'],
                                                   C_storage[time]['Serpentinites_raw'],
                                                   C_storage[time]['Serpentinites_raw_vertical'],
                                                   C_storage[time]['Thickness'],
                                                   C_storage[time]['Vertical Area'],
                                                   C_storage[time]['Spreading Rate']):


        a = s[0] #total thick
        b = s[1] #upper thick
        c = s[2] #lower thick
        d = s[3] #trans thick
        e = s[4] #dyke thick
        f = s[5] #gabbro thick
        
        #if time > 160:
        #    #print 'here'
        #    #use vertical values if old, gets undone in subduction flux, but not perfectly
        #    #this lets us detect them later on more easily
        #    #divide by two so we don't double dip
#
        #    o = p * 1000000/2    #C
        #    q = r * 1000000/2    #serp
        #    a = t[0] * 1000000/2 #total thick
        #    b = t[1] * 1000000/2 #upper thick
        #    c = t[2] * 1000000/2 #lower thick
        #    d = t[3] * 1000000/2 #trans thick
        #    e = t[4] * 1000000/2 #dyke thick
        #    f = t[5] * 1000000/2 #gabbro thick
            
        points = []
        points.append((i,k))
        points.append((j,l))
        polyline = pygplates.PolylineOnSphere(points)
        #get stage rotations
        line_rotation_right = rotation_model.get_rotation(next_timestep, int(m), time)
        line_rotation_left = rotation_model.get_rotation(next_timestep, int(n), time)
        reconstructed_polyline_right = line_rotation_right * polyline
        reconstructed_polyline_left = line_rotation_left * polyline
        tessellated_polyline_right = reconstructed_polyline_right.to_tessellated(np.radians(2))
        tessellated_polyline_left = reconstructed_polyline_left.to_tessellated(np.radians(2))
        latlon_list_right = tessellated_polyline_right.to_lat_lon_list()
        latlon_list_left = tessellated_polyline_left.to_lat_lon_list()
        recon_lats_right, recon_lons_right = [i[0] for i in latlon_list_right], [i[1] for i in latlon_list_right]
        recon_lats_left, recon_lons_left = [i[0] for i in latlon_list_left], [i[1] for i in latlon_list_left]
        
        if time == np.max(times):
            #print h
            init_curr_tessellated_polyline_right = polyline.to_tessellated(np.radians(2))
            init_curr_tessellated_polyline_left = polyline.to_tessellated(np.radians(2))
            init_curr_latlon_list_right = init_curr_tessellated_polyline_right.to_lat_lon_list()
            init_curr_latlon_list_left = init_curr_tessellated_polyline_left.to_lat_lon_list()
            init_curr_recon_lats_right, init_curr_recon_lons_right = [i[0] for i in init_curr_latlon_list_right], [i[1] for i in init_curr_latlon_list_right]
            init_curr_recon_lats_left, init_curr_recon_lons_left = [i[0] for i in init_curr_latlon_list_left], [i[1] for i in init_curr_latlon_list_left]

            for i in range(len(init_curr_recon_lats_right)):
                init_curr_lat.append(init_curr_recon_lats_right[i])
                init_curr_lat.append(init_curr_recon_lats_left[i])
                init_curr_lon.append(init_curr_recon_lons_right[i])
                init_curr_lon.append(init_curr_recon_lons_left[i])
                #init_curr_plateIDs.append(m)
                #init_curr_plateIDs.append(n)
                #update arrays with data at new timestep
                #right then left
        for i in range(len(recon_lats_right)):
            MOR_lat.append(recon_lats_right[i])
            MOR_lat.append(recon_lats_left[i])
            MOR_lon.append(recon_lons_right[i])
            MOR_lon.append(recon_lons_left[i])
            #have to do zvals twice, once for right ridge, second time for left ridge
            MOR_zvals.append([o,q,a,b,c,d,e,f,u,time])
            MOR_zvals.append([o,q,a,b,c,d,e,f,u,time])
            #plateIDs as well
            MOR_plateIDs.append(int(m))
            MOR_plateIDs.append(int(n))
    if time == np.max(times):
        #to initialise at max time, we get the partitioner and plateIDs of the first step of rotated points
        #this isn\'t technically correct, but as we are only dealing with 1 Ma worth of points, they should fall
        #within the correct ocean basin unless at initialisation time the ridge is being subducted (not in our case)
        curr_plate_partitioner = pygplates.PlatePartitioner(curr_resolved_topologies, rotation_model)
        curr_plateIDs = MOR_plateIDs
        curr_lat, curr_lon = init_curr_lat, init_curr_lon
        next_lat = (MOR_lat)
        next_lon = (MOR_lon)
        next_zvals = (MOR_zvals)
    if time != np.max(times):
        print 'whats been added', len(MOR_lat)
        next_lat.extend(MOR_lat)
        next_lon.extend(MOR_lon)
        next_zvals.extend(MOR_zvals)
        
            
        print len(next_lat)
        #print len(next_lat)

    print("--- %s seconds for timestep ---" % (tme.time() - start_time))
print("--- %s seconds overall ---" % (tme.time() - start_time))


350
--- 4.96224308014 seconds for timestep ---
349
first 710
second 710
third 696
whats been added 694
1390
--- 9.91252994537 seconds for timestep ---
348
first 1390
second 1390
third 1346
whats been added 744
2090
--- 14.7954111099 seconds for timestep ---
347
first 2090
second 2090
third 2057
whats been added 742
2799
--- 19.863270998 seconds for timestep ---
346
first 2799
second 2799
third 2769
whats been added 740
3509
--- 25.0220620632 seconds for timestep ---
345
first 3509
second 3509
third 3477
whats been added 752
4229
--- 30.3048849106 seconds for timestep ---
344
first 4229
second 4229
third 4191
whats been added 782
4973
--- 35.8861000538 seconds for timestep ---
343
first 4973
second 4973
third 4930
whats been added 786
5716
--- 41.6499209404 seconds for timestep ---
342
first 5716
second 5716
third 5684
whats been added 786
6470
--- 47.1312360764 seconds for timestep ---
341
first 6470
second 6470
third 6423
whats been added 792
7215
--- 53.1573851109 seconds for timeste

first 50068
second 50068
third 49903
whats been added 662
50565
--- 713.668546915 seconds for timestep ---
274
first 50565
second 50565
third 49953
whats been added 572
50525
--- 727.09433198 seconds for timestep ---
273
first 50525
second 50525
third 50345
whats been added 566
50911
--- 740.324103117 seconds for timestep ---
272
first 50911
second 50911
third 50736
whats been added 568
51304
--- 753.688946962 seconds for timestep ---
271
first 51304
second 51304
third 51141
whats been added 566
51707
--- 767.066896915 seconds for timestep ---
270
first 51707
second 51707
third 51473
whats been added 718
52191
--- 781.281183004 seconds for timestep ---
269
first 52191
second 52191
third 51924
whats been added 648
52572
--- 795.66801405 seconds for timestep ---
268
first 52572
second 52572
third 51494
whats been added 650
52144
--- 809.938699007 seconds for timestep ---
267
first 52144
second 52144
third 51097
whats been added 640
51737
--- 824.325751066 seconds for timestep ---
266
fir

first 41645
second 41645
third 41310
whats been added 480
41790
--- 1582.43391609 seconds for timestep ---
200
first 41790
second 41790
third 40959
whats been added 494
41453
--- 1593.55480599 seconds for timestep ---
199
first 41453
second 41453
third 41037
whats been added 510
41547
--- 1604.93811512 seconds for timestep ---
198
first 41547
second 41547
third 41191
whats been added 508
41699
--- 1616.40171313 seconds for timestep ---
197
first 41699
second 41699
third 41369
whats been added 512
41881
--- 1627.94083405 seconds for timestep ---
196
first 41881
second 41881
third 41508
whats been added 510
42018
--- 1639.46718311 seconds for timestep ---
195
first 42018
second 42018
third 41533
whats been added 518
42051
--- 1651.11311913 seconds for timestep ---
194
first 42051
second 42051
third 41644
whats been added 508
42152
--- 1662.84001899 seconds for timestep ---
193
first 42152
second 42152
third 41755
whats been added 514
42269
--- 1674.85106301 seconds for timestep ---
192
f

first 80203
second 80203
third 79364
whats been added 2112
81476
--- 2666.82066894 seconds for timestep ---
126
first 81476
second 81476
third 80584
whats been added 2096
82680
--- 11716.771735 seconds for timestep ---
125
first 82680
second 82680
third 82377
whats been added 2252
84629
--- 11740.0640721 seconds for timestep ---
124
first 84629
second 84624
third 83366
whats been added 2340
85706
--- 11765.0442901 seconds for timestep ---
123
first 85706
second 85701
third 84572
whats been added 2342
86914
--- 11788.9526081 seconds for timestep ---
122
first 86914
second 86912
third 86031
whats been added 2336
88367
--- 11811.5984721 seconds for timestep ---
121
first 88367
second 88365
third 87752
whats been added 2336
90088
--- 11834.6876919 seconds for timestep ---
120
first 90088
second 90088
third 89095
whats been added 1998
91093
--- 11858.467037 seconds for timestep ---
119
first 91093
second 91093
third 90397
whats been added 1966
92363
--- 11881.500844 seconds for timestep ---

first 171635
second 171635
third 169374
whats been added 2296
171670
--- 14049.7209311 seconds for timestep ---
54
first 171670
second 171670
third 171443
whats been added 2306
173749
--- 14091.3918471 seconds for timestep ---
53
first 173749
second 173749
third 173459
whats been added 2400
175859
--- 14133.290905 seconds for timestep ---
52
first 175859
second 175859
third 175659
whats been added 2414
178073
--- 14175.2689371 seconds for timestep ---
51
first 178073
second 178073
third 176864
whats been added 1958
178822
--- 14217.7481771 seconds for timestep ---
50
first 178822
second 178822
third 177646
whats been added 2030
179676
--- 14259.1022229 seconds for timestep ---
49
first 179676
second 179676
third 178999
whats been added 2158
181157
--- 14301.3710001 seconds for timestep ---
48
first 181157
second 181157
third 180515
whats been added 2230
182745
--- 14343.669802 seconds for timestep ---
47
first 182745
second 182745
third 181984
whats been added 2244
184228
--- 14386.070